In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import itertools
import json
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import utils as ut
import LIM_class
from LSTM_enc_dec import *

plt.style.use("../plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
data = xr.open_dataset("./data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data_old = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]
mask = xr.open_dataset("./data/sftlf_fx_CESM2_historical_r1i1p1f1.nc")["sftlf"]

#14400 orginial size
data = data[:, :, :]

data = ut.apply_mask(mask, data)
#print("Data : {} + shape {}".format(data, data.shape))

data_anomalies = ut.calculate_monthly_anomalies(data)
#print("Month mean : {} + shape : {}".format(data_anomalies, data_anomalies.shape))

data_cropped =ut.crop_xarray(data_anomalies)
#print("Data cropped : {} + shape : {}".format(data_cropped, data_cropped.shape))


pca_10 = ut.SpatioTemporalPCA(data_cropped, n_components=20)
#pca_10 = ut.SpatioTemporalPCA(data_anomalies, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

In [11]:
from LSTM_model import *
#
# # Set random seed for reproducibility
# torch.manual_seed(42)
#
def hyperparameter_training_loop(data, hyperparams):

    # Generate all possible combinations of hyperparameters
    combinations = list(itertools.product(*hyperparams.values()))

    overall_best_model = {"hidden_size": None,
                         "learning_rate": None,
                         "num_epochs": None,
                         "loss": float('inf')}

    # Iterate over each hyperparameter combination
    for i, params in enumerate(combinations):
        print(f"Testing parameter combination {i+1}/{len(combinations)}: {params}")

        # Create a new model with the current hyperparameters
        hidden_size = params[0]
        learning_rate = params[1]
        num_epochs = params[2]
        num_layers = params[3]
        sequence_length = params[4]
        batch_size = params[5]

        dataloader = create_dataloader(data, sequence_length=sequence_length, batch_size=batch_size, shuffle=False)

        model = LSTMNetwork(1, hidden_size, num_layers)
        losses = train(model, dataloader, num_epochs, learning_rate)

        # Save the model and hyperparameters to a file
        result = {
            'hyperparameters': {
                'hidden_size': hidden_size,
                'learning_rate': learning_rate,
                'num_epochs': num_epochs,
                'num_layers': num_layers,
                'sequence_length': sequence_length,
                'best_loss': losses[-1],
                "losses": losses
            }
        }
        if losses[-1] < overall_best_model["loss"]:
            overall_best_model["hidden_size"] = params[0]
            overall_best_model["learning_rate"] = params[1]
            overall_best_model["num_epochs"] = params[2]
            overall_best_model["loss"] = losses[-1]
            overall_best_model["losses"] = losses

        filename = f"./model_trained_lstm/fnn_model_{i+1}.pt"
        with open(filename, 'w') as f:
            json.dump(result, f)

        print(f"Saved model and hyperparameters to {filename}\n")

    return overall_best_model
#
#
# # Create the DataLoader for first principal component
# data = np.array(data)[0]
# print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))
#
# # Hyperparameter search space
# hyperparams = {
#     'hidden_size': [32],
#     'learning_rate': [0.001, 0.002],
#     'num_epochs': [1000],
#     'num_layers': [3],
#     'sequence_length': [5],
#     'batch_size': [1]
# }
#
# #Train the model
# #train(model, dataloader, num_epochs, learning_rate)
# best_model = hyperparameter_training_loop(data, hyperparams)
#
# print(f"Best model: {best_model}")
# plot_loss_evolution(best_model["losses"], np.arange(0,best_model["num_epochs"], 100, dtype=int), best_model["learning_rate"], best_model["hidden_size"])

In [12]:
# LSTM encoder-decoder

# Set random seed for reproducibility
torch.manual_seed(42)

# Create the DataLoader for first principal component
data = pca_10.principal_components()
data = np.array(data)
print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))

# Hyperparameter search space
hyperparams = {
    'hidden_size': [32],
    'learning_rate': [0.001, 0.002],
    'num_epochs': [1000],
    'num_layers': [3],
    'sequence_length': [5],
    'batch_size': [1]
}

#Train the model

index_train = int(0.8 * len(data[0, :]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

input_data, target_data = windowed_dataset(data_train, input_window=5, output_window=12)
input_data_test, target_data_test = windowed_dataset(data_test)

print("Input data : {} + shape: {} + type: {}".format(input_data, input_data.shape, type(input_data)))



# convert windowed data from np.array to PyTorch tensor
X_train, Y_train, X_test, Y_test = numpy_to_torch(input_data, target_data, input_data_test, target_data_test)

# specify model parameters and train
model = LSTM_seq2seq(input_size = X_train.shape[2], hidden_size = 32)
loss = model.train_model(X_train, Y_train, n_epochs = 500, target_len = 12, batch_size = 32, training_prediction = 'recursive', teacher_forcing_ratio = 0.6, learning_rate = 0.01, dynamic_tf = False)
print("Loss : {} + shape: {} + type: {}".format(loss, loss.shape, type(loss)))

Data : [[  3.082717     7.58872     11.522211   ...   2.1110816   -2.2536314
  -10.422921  ]
 [ 15.300908    13.185972     6.481787   ...  26.23216     23.710655
   19.59892   ]
 [ -1.5274546    4.64411      4.499988   ...  -9.4210825  -11.31246
  -15.245653  ]
 ...
 [  0.07792819   1.0995704    1.4193639  ...  -1.4664601    1.7521834
   -2.5803611 ]
 [  0.8366144    2.314835     1.0267301  ...   0.29932857  -0.27528232
   -0.15596846]
 [  0.8229446    0.4607976    0.8099255  ...  -1.9031905   -1.2379901
   -1.0049288 ]] + shape: (20, 14400) + type: <class 'numpy.ndarray'>
Input data : [[[ 3.08271694e+00  7.58871984e+00  1.15222111e+01  2.02901573e+01
    2.70137196e+01  3.14075832e+01  4.32124710e+01  5.43983459e+01
    6.27646065e+01  6.35853958e+01  6.44254684e+01  7.03778000e+01
    7.13412628e+01  6.47799149e+01  5.62321281e+01  5.41466408e+01
    4.06710472e+01  2.21597424e+01 -1.01463091e+00 -1.78049965e+01]
  [ 1.53009081e+01  1.31859722e+01  6.48178720e+00  2.30049849e+00
    

  0%|          | 0/500 [00:00<?, ?it/s]


ZeroDivisionError: float division by zero